In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [ ]:
sample = pd.read_csv('/content/drive/MyDrive/alcups2024/sample_submission.csv', sep='\t')
sample

,idx,Score
0,13999,Positive
1,14000,Positive
2,14001,Positive
3,14002,Positive
4,14003,Positive
...,...,...
5995,19994,Positive
5996,19995,Positive
5997,19996,Positive
5998,19997,Positive


In [ ]:
sample['idx']

,idx
0,13999
1,14000
2,14001
3,14002
4,14003
...,...
5995,19994
5996,19995
5997,19996
5998,19997


In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/alcups2024/train.csv', sep='\t')
test_data = pd.read_csv('/content/drive/MyDrive/alcups2024/test.csv', sep='\t')

In [ ]:
train_data

,idx,Score,Text
0,0,Positive,В Альфа-Банке работает замечательная девушка -...
1,1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...
2,2,Positive,Очень порадовала оперативность работы в банке....
3,3,Negative,Имела неосторожность оформить потреб. кредит в...
4,4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...
...,...,...,...
13994,13994,Positive,"О высокой надёжности МКБ, порядочности и добро..."
13995,13995,Positive,"Обслуживаюсь в офисе на Чернореченской 42а, ка..."
13996,13996,Positive,Попала сегодня в очень неприятную ситуацию. Ре...
13997,13997,Positive,Добрый день! Давно являюсь клиентом банка Русс...


In [ ]:
test_data

,idx,Text
0,13999,Замечательная история об открытии счета в банк...
1,14000,"23.07.14г. я досрочно внес сумму, указанную оп..."
2,14001,Была задолженность по кредитной карте. Банк по...
3,14002,Срочно понадобилась определенная денежная сумм...
4,14003,Здравствуйте. В местном отделении Бинбанка г. ...
...,...,...
5995,19994,Уважаемые представители Русфинанс Банка! Прошу...
5996,19995,04 июня сняли с дебетовой карты банка 10 000 T...
5997,19996,В апреле решил я купить шуруповерт. Посмотрел ...
5998,19997,Недавно взял потребкредит в этом банке. Все че...


In [ ]:
def preprocess_text(text):
    text = text.lower()
    return text

train_data['Text'] = train_data['Text'].apply(preprocess_text)
test_data['Text'] = test_data['Text'].apply(preprocess_text)

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_data['Text'])
y_train = train_data['Score']

X_test = vectorizer.transform(test_data['Text'])

In [ ]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [ ]:
predictions = model.predict(X_test)

# Оценка точности на test_data
# Для этого обычно нужно иметь метки, их можно не иметь, если вы работаете на неизвестных данных
# Для оценки на train_data, можно использовать train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Обучение на новой выборке
model.fit(X_train, y_train)
val_predictions = model.predict(X_val)

# Оценка accuracy
accuracy = accuracy_score(y_val, val_predictions)
print(f"Accuracy: {accuracy:.4f}")

# Расчет баллов
points = 400 * max(0, (accuracy * 2 - 1))
print(f"Points: {points:.2f}")


Accuracy: 0.9330
Points: 346.43


In [ ]:
submission = pd.DataFrame({'idx': sample['idx'], 'Score': predictions})

In [ ]:
submission

,idx,Score
0,13999,Negative
1,14000,Negative
2,14001,Negative
3,14002,Negative
4,14003,Negative
...,...,...
5995,19994,Negative
5996,19995,Positive
5997,19996,Negative
5998,19997,Positive


In [ ]:
submission.to_csv('submissionz1.csv', index=False, sep='\t')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np

# Загрузка данных
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Предобработка данных
X = train_data.drop(columns=['salary_mean_net'])
y = train_data['salary_mean_net']

# Если в данных есть текстовые поля, применяем векторизацию
X = pd.get_dummies(X)  # Пример, можно использовать TF-IDF или другие методы

# Разделяем данные на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)

# Прогнозирование на валидационной выборке
y_pred = model.predict(X_val)

# Оценка модели
mape = mean_absolute_percentage_error(y_val, y_pred)
print(f'MAPE: {mape * 100:.2f}%')

# Прогнозирование на тестовых данных
X_test = test_data.drop(columns=['id'])  # Убедитесь, что столбцы соответствуют
X_test = pd.get_dummies(X_test)  # Примените ту же обработку

y_test_pred = model.predict(X_test)

# Создание файла для submission
submission = pd.DataFrame({'id': test_data['id'], 'salary_mean_net': y_test_pred})
submission.to_csv('submission.csv', index=False)
